In [2]:
from algorithms.iterative_relaxation import IterativeRelaxation
from algorithms.decision_procedure import MarabouCoreDP
from algorithms.layer_prop_decision_tree import LayerPropDecisionTree
from models.test_models import ProphecyPaperNetwork, TestModel
import torch
from torch import nn
import numpy as np
import pandas as pd  

ModuleNotFoundError: No module named 'algorithms'

### Labeled dataset for Prophecy paper's model

In [ ]:
# we'll consider a range of -50 to 50 for our inputs x1 and x2. 
# We'll iterate through all possible pairs of (x1, x2) within this range, with increments of 1
model = ProphecyPaperNetwork()
_act_handles, activation_signature = model.attach_relu_activation_hook()
_out_handles, layer_outputs = model.attach_layer_output_hook()


def satisfies_postcond(output: list, postcond: int):
  # if postcond is y1, then y1 must be greater than every other class. 
  postcond_var = output[postcond]
  result = True
  for index, val in enumerate(output):
    if index == postcond: continue
    result = result and (postcond_var > val)
  result = 1 if result == True else -1
  return result

def create_training_df(inputs, postcond: int):
  data = []
  for input_data in inputs:
    X = torch.tensor([input_data], dtype=torch.float)
    logits = model(X)
    prediction_prob = nn.Softmax(dim=1)(logits)
    y_pred = prediction_prob.argmax(1)
    output = layer_outputs["final_output"][0].tolist()
    
    data_point = { 
      "input": input_data, 
      "output": output, 
      "satisfies_postcon": satisfies_postcond(output, postcond) 
    }
    for name, layer_activation in activation_signature.items():
      data_point[name] = [1 if val == "ON" else 0 for val in layer_activation]
    data.append(data_point)
    
  return pd.DataFrame(data)

In [ ]:
# create input pairs for model
inputs = []
for x1 in np.arange(-50, 51, 1):
  for x2 in np.arange(-50, 51, 1):
    inputs.append([x1, x2])

df = create_training_df(inputs, 0)
print(len(df))
df.head(10)

### Decision tree learning to find potential layer properties

In [ ]:
decision_tree = LayerPropDecisionTree(df, X_col="linear_relu_stack.1", Y_col="satisfies_postcon")
decision_tree.get_potential_layer_properties()

In [ ]:
decision_tree.visualize_tree()